In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

In [2]:
df = pd.read_csv('data/kc_house_data.csv')

# Data Cleaning

Let's look at the data:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

From top down, several initial transformations must take place.

Date: change to date time

Waterfront: inspect NaNs (and likely drop)
Other columns that require further inspection:
    -View, Condition, Grade, Year Renovated

Sqft Basement: convert to float

In [4]:
df['sqft_basement'].unique()

array(['0.0', '400.0', '910.0', '1530.0', '?', '730.0', '1700.0', '300.0',
       '970.0', '760.0', '720.0', '700.0', '820.0', '780.0', '790.0',
       '330.0', '1620.0', '360.0', '588.0', '1510.0', '410.0', '990.0',
       '600.0', '560.0', '550.0', '1000.0', '1600.0', '500.0', '1040.0',
       '880.0', '1010.0', '240.0', '265.0', '290.0', '800.0', '540.0',
       '710.0', '840.0', '380.0', '770.0', '480.0', '570.0', '1490.0',
       '620.0', '1250.0', '1270.0', '120.0', '650.0', '180.0', '1130.0',
       '450.0', '1640.0', '1460.0', '1020.0', '1030.0', '750.0', '640.0',
       '1070.0', '490.0', '1310.0', '630.0', '2000.0', '390.0', '430.0',
       '850.0', '210.0', '1430.0', '1950.0', '440.0', '220.0', '1160.0',
       '860.0', '580.0', '2060.0', '1820.0', '1180.0', '200.0', '1150.0',
       '1200.0', '680.0', '530.0', '1450.0', '1170.0', '1080.0', '960.0',
       '280.0', '870.0', '1100.0', '460.0', '1400.0', '660.0', '1220.0',
       '900.0', '420.0', '1580.0', '1380.0', '475.0', 

In [5]:
# Dat to datetime
df['date'] = pd.to_datetime(df['date'])

# SQFT Basement to float
df.drop(df.loc[df['sqft_basement'] == '?'].index, inplace=True)
df['sqft_basement'] = df['sqft_basement'].astype(float)

Now let's look at values for the other colummns to see if there's anything obvious for us to drop

First just the columns with categorical or narrowly discrete units

In [6]:
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [7]:
df['bedrooms'].unique()

# Drop entry with 33 Bedrooms
df.drop(df.loc[df['bedrooms'] == 33].index, inplace=True)

df['bedrooms'].unique()

array([ 3,  2,  4,  5,  1,  6,  7,  8,  9, 11, 10])

In [8]:
df['bathrooms'].unique()

# Drop entry with 0.5 bathrooms
df.drop(df.loc[df['bathrooms'] == 0.5].index, inplace=True)

df['bathrooms'].unique()

array([1.  , 2.25, 3.  , 2.  , 4.5 , 1.5 , 2.5 , 1.75, 2.75, 3.25, 4.  ,
       3.5 , 0.75, 4.75, 5.  , 4.25, 3.75, 1.25, 5.25, 6.  , 5.5 , 6.75,
       5.75, 8.  , 7.5 , 7.75, 6.25, 6.5 ])

In [9]:
df['floors'].unique()

array([1. , 2. , 1.5, 3. , 2.5, 3.5])

In [10]:
df['waterfront'].unique()
df['waterfront'].value_counts(dropna=False)

NO     18658
NaN     2339
YES      142
Name: waterfront, dtype: int64

Although there are over 2,000 counts of NaN for waterfront, I feel fairly comfortable turning these NaNs to "No"s

In [11]:
# Turn NaNs to No
df['waterfront'] = df['waterfront'].fillna('NO')

# Convert to 0 or 1
mapdict = {'YES': 1, 'NO': 0}
df['waterfront'] = df['waterfront'].map(mapdict)

# Check result
df['waterfront'].value_counts(dropna=False)

0    20997
1      142
Name: waterfront, dtype: int64

In [12]:
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [13]:
df['view'].value_counts()

NONE         19014
AVERAGE        930
GOOD           496
FAIR           327
EXCELLENT      311
Name: view, dtype: int64

In [14]:
# Create Yes/No column for view
df['view_y_n'] = df['view'].apply(lambda x: 0 if x == 'NONE' else 1)
df['view_y_n'].value_counts()

# Actually let's just drop view because so many are None.
df.drop(['view_y_n', 'view'], axis = 1, inplace=True)

df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [15]:
df['condition'].value_counts()

# Map numerical values

mapdict = {'Poor': 0, 'Fair': 1, 'Average': 2, 'Good': 3, 'Very Good': 4}
df['condition'] = df['condition'].map(mapdict)


df['condition'].value_counts()


2    13724
3     5556
4     1665
1      166
0       28
Name: condition, dtype: int64

In [16]:
df['grade'].value_counts()

# Let's leave this for now

7 Average        8787
8 Good           5933
9 Better         2557
6 Low Average    1995
10 Very Good     1112
11 Excellent      391
5 Fair            234
12 Luxury          89
4 Low              27
13 Mansion         13
3 Poor              1
Name: grade, dtype: int64

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21139 entries, 0 to 21596
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             21139 non-null  int64         
 1   date           21139 non-null  datetime64[ns]
 2   price          21139 non-null  float64       
 3   bedrooms       21139 non-null  int64         
 4   bathrooms      21139 non-null  float64       
 5   sqft_living    21139 non-null  int64         
 6   sqft_lot       21139 non-null  int64         
 7   floors         21139 non-null  float64       
 8   waterfront     21139 non-null  int64         
 9   condition      21139 non-null  int64         
 10  grade          21139 non-null  object        
 11  sqft_above     21139 non-null  int64         
 12  sqft_basement  21139 non-null  float64       
 13  yr_built       21139 non-null  int64         
 14  yr_renovated   17385 non-null  float64       
 15  zipcode        2113

In [19]:
df.iloc[2]

id                        5631500400
date             2015-02-25 00:00:00
price                         180000
bedrooms                           2
bathrooms                          1
sqft_living                      770
sqft_lot                       10000
floors                             1
waterfront                         0
condition                          2
grade                  6 Low Average
sqft_above                       770
sqft_basement                      0
yr_built                        1933
yr_renovated                     NaN
zipcode                        98028
lat                          47.7379
long                        -122.233
sqft_living15                   2720
sqft_lot15                      8062
Name: 2, dtype: object

In [ ]:
import seaborn as sns

sns.displot(df['price']);


In [ ]:
df['yr_renovated'].unique()


In [ ]:
df['bedrooms'].unique()

In [ ]:
df.loc[df['bedrooms'] == 33]

In [ ]:
df['yr_built'].unique()